In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pattern
from pattern.text.en import singularize

In [ ]:
reviews:pd.DataFrame=pd.read_csv("mcdonalds-yelp-negative-reviews.csv",encoding='latin1')

**Used regex cleaning and substitution**

In [ ]:
AH_PATTERN = re.compile(r'\bA{1,20}H{1,20}\b', flags=re.IGNORECASE)
uh_PATTERN = re.compile(r'\bU{1,20}H{1,20}\b', flags=re.IGNORECASE)
ew_PATTERN = re.compile(r'\bE{1,20}W{1,20}\w+\b', flags=re.IGNORECASE)
Time_pattern=re.compile(r'\b(\d+(?::\d+)?(?:/\d+)?(?::\d+)?[\s]?(?:A|P)M)\b', flags=re.IGNORECASE)
noise_number=re.compile(r'(\$?\d+(?:\w+)?)')
underscore=re.compile(r'(\b.?_\w+?)')
ugh_PATTERN = re.compile(r'\bU{1,20}G{1,20}h{0,20}\b', flags=re.IGNORECASE)
slow_PATTERN = re.compile(r'\bS{1,20}L{1,20}O{1,20}W{1,20}\b', flags=re.IGNORECASE)
too_PATTERN = re.compile(r'\bT{1,20}O{1,20}\b', flags=re.IGNORECASE)
so_PATTERN = re.compile(r'\bS{1,20}O{1,20}\b', flags=re.IGNORECASE)
yo_PATTERN = re.compile(r'\by(?:e|o)(?:a|p)[h]?\b', flags=re.IGNORECASE)
all_PATTERN = re.compile(r'\bA{1,20}L{1,20}\b', flags=re.IGNORECASE)
top_PATTERN = re.compile(r'\btop[s]?(?:ped|ping)?\b', flags=re.IGNORECASE)
still_PATTERN = re.compile(r'\bstil{1,20}\b', flags=re.IGNORECASE)
wait_PATTERN = re.compile(r'\bwait(?:ing|ed)?\b', flags=re.IGNORECASE)
spill_PATTERN = re.compile(r'\bspill(?:ing|ed)?\b', flags=re.IGNORECASE)
gr_PATTERN = re.compile(r'\bG{1,20}R{1,20}\b', flags=re.IGNORECASE)
spend_PATTERN = re.compile(r'\bspen(?:t|ding)?\b', flags=re.IGNORECASE)
why_PATTERN = re.compile(r'\bwhy{1,20}\b', flags=re.IGNORECASE)
way_PATTERN = re.compile(r'\bway{1,20}\b', flags=re.IGNORECASE)
woo_PATTERN = re.compile(r'\bwo{1,20}\b', flags=re.IGNORECASE)
grill_PATTERN = re.compile(r'\bgrill(?:ed|ing)?\b', flags=re.IGNORECASE)
worry_PATTERN = re.compile(r'\bworr(?:y|ied)?\b', flags=re.IGNORECASE)
work_PATTERN = re.compile(r'\bwork(?:ed|ing)?\b', flags=re.IGNORECASE)

In [ ]:
reviews['review'] = reviews.review.str.replace(AH_PATTERN, '_ahhh_token_')
reviews['review'] = reviews.review.str.replace(uh_PATTERN, 'uh')
reviews['review'] = reviews.review.str.replace(Time_pattern, 'time')
reviews['review'] = reviews.review.str.replace(noise_number, 'number')
reviews['review'] = reviews.review.str.replace(underscore, 'underscore')
reviews['review'] = reviews.review.str.replace(ugh_PATTERN, 'ugh')
reviews['review'] = reviews.review.str.replace('wikipedium', 'wikipedia')
reviews['review'] = reviews.review.str.replace('yum', 'yummy')
reviews['review'] = reviews.review.str.replace(slow_PATTERN, 'slow')
reviews['review'] = reviews.review.str.replace(ew_PATTERN, 'ew')
reviews['review'] = reviews.review.str.replace(too_PATTERN, 'too')
reviews['review'] = reviews.review.str.replace(so_PATTERN, 'so')
reviews['review'] = reviews.review.str.replace(yo_PATTERN, 'y')
reviews['review'] = reviews.review.str.replace(all_PATTERN, 'all')
reviews['review'] = reviews.review.str.replace(top_PATTERN, 'top')
reviews['review'] = reviews.review.str.replace(still_PATTERN, 'still')
reviews['review'] = reviews.review.str.replace(wait_PATTERN, 'wait')
reviews['review'] = reviews.review.str.replace('thorugh', 'thorough')
reviews['review'] = reviews.review.str.replace('thrown', 'throw')
reviews['review'] = reviews.review.str.replace(spill_PATTERN, 'spill')
reviews['review'] = reviews.review.str.replace('tastele', 'tasteless')
reviews['review'] = reviews.review.str.replace('tasteles', 'tasteless')
reviews['review'] = reviews.review.str.replace(gr_PATTERN, 'gr')
reviews['review'] = reviews.review.str.replace('tastey', 'tasty')
reviews['review'] = reviews.review.str.replace('tasted', 'taste')
reviews['review'] = reviews.review.str.replace(spend_PATTERN, 'spend')
reviews['review'] = reviews.review.str.replace(why_PATTERN, 'why')
reviews['review'] = reviews.review.str.replace(way_PATTERN, 'way')
reviews['review'] = reviews.review.str.replace(woo_PATTERN, 'woo')
reviews['review'] = reviews.review.str.replace(grill_PATTERN, 'grill')
reviews['review'] = reviews.review.str.replace(worry_PATTERN, 'worry')
reviews['review'] = reviews.review.str.replace(work_PATTERN, 'work')
reviews['review'] = reviews.review.str.replace('saw', 'see')

In [ ]:
reviews=reviews['review']

**Add some stopwords to remove useless and redundant words**

In [ ]:
stopwords=stopwords.words('english')

In [ ]:
stopwords = set(stopwords + [".",'.', ",",":", "''", "'s", "'", "``", "(", ")", "-",'!','?',"*",'actually','usually','really','mcds','''mcdonald's''',
               'mcdonald','mcdonalds','''i'm''','''i've''','_ahhh_token_','time','number','say','alway',
            'abbreviate','able','abour','underscore','î_','uh','ew','gr','woo','thi','wa','ca', "n't"])

In [ ]:
cleaned_reviews = []
for review in reviews:
    words = review.split()
    new_words = []
    for word in words:
        word=singularize(word).lower()
        if word.strip().lower() not in stopwords:
            new_words.append(word)
    cleaned_review = " ".join(new_words)
    cleaned_reviews.append(cleaned_review)

**Lemmatization with pos tag code is coming from gaurav5430's Github Gist**

**The reason** I used lemmatization is we are doing a review text analysis and I hope the result of this analysis can more precisely reflect the true sentiment of the reviews. Using stemming may increase false positive rate and reduce precision since stemming  is trying to chop off the ends of words and will count some words which have different meanings into same root. Lemmatization will return the base or dictionary form of a word, which means it can match words more precisely.

In [ ]:
lemmatizer = WordNetLemmatizer()
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [ ]:
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
df_lemma=[]
for i in cleaned_reviews:
    lemma_s=lemmatize_sentence(i)
    df_lemma.append(lemma_s)

In [ ]:
# after lemmatizing, some words will change from non-stopword to stopwords -> abbreviated ->abbreviate
# so I remove stopwords again after lemmatizing
final_reviews = []
for review in df_lemma:
    words2 = review.split()
    new_words2 = []
    for word2 in words2:
        word2=singularize(word2)
        if word2.strip().lower() not in stopwords:
            new_words2.append(word2)
    final_reviews.append(new_words2)

**Used 2 for my 2-ngrams**

In [ ]:
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures

collocation_finder = BigramCollocationFinder.from_documents(final_reviews)
measures = BigramAssocMeasures()

collocation_finder.nbest(measures.raw_freq, 25)

**Generated TF-IDF**

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(2,2),
                             token_pattern=r'\b[a-zA-Z]{3,}\b',
                             max_df=5, stop_words=stopwords)

In [ ]:
corpus= []
for review in df_lemma:
    words2 = review.split()
    new_words2 = []
    for word2 in words2:
        word2=singularize(word2)
        if word2.strip().lower() not in stopwords:
            new_words2.append(word2)
    final_review = " ".join(new_words2)
    corpus.append(final_review)

In [ ]:
X = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)

In [ ]:
tf_idf =pd.DataFrame(tf_idf.sum(axis=1),columns=['tf_idf'])

In [ ]:
tf_idf.sort_values(by="tf_idf", ascending=False, inplace=True)

In [ ]:
top20=tf_idf[:21]

In [ ]:
top20.reset_index(inplace=True)

In [ ]:
top20.sort_values(by="tf_idf",ascending=True,inplace=True)

**The features** my analysis showed that customers cited as reasons for a poor review:

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))
plt.barh(top20['index'],top20['tf_idf'])
plt.yticks(rotation = 'horizontal',fontsize=15)
plt.show()

**Check for the food key word hot mustard and dip cone**

In [ ]:
reviews[list(map(lambda line:bool(re.search(r'dip(?:ped)?\scones?',line,flags=re.IGNORECASE)),reviews))]

In [ ]:
print(reviews[162])
print(reviews[384])
print(reviews[804])
print(reviews[1067])
print(reviews[1106])

In [ ]:
reviews[list(map(lambda line:bool(re.search(r'hot\smustard?',line,flags=re.IGNORECASE)),reviews))]

In [ ]:
print(reviews[367])
print(reviews[493])
print(reviews[613])

**The most common issues identified from my analysis that generated customer dissatisfaction are:**

* service is rude and slow; 
* customers need wait long time; 
* food is not clean and customers may have stomach ache after eating; 
* McDonalds' food make customers gain fat

Recommendations: improve customer service and speed up the service; try to make clean and healthy food.

## Product Attribution (Feature Engineering and Regex Practice)

In [ ]:
data:pd.DataFrame=pd.read_csv("truncated_catalog.csv")

In [ ]:
data['gender_info']=data.iloc[:,[1,2,3]].apply(lambda x:' '.join(x.astype(str)),axis=1)

**Filter for only women's clothing items**.

In [ ]:
data['is_womens_clothing']=list(map(lambda line:bool(re.search(
    r'(wom(?:a|e)n|girl(?:friend)?s?|female|wifes?|wives|(?:grand)?daughters?|(?:grand)?monthers?|(?:grand)?mom(?:my)?s?|sisters?|nieces?|unisex)',
                      line,flags=re.IGNORECASE)),data['gender_info']))

In [ ]:
data.drop('gender_info',inplace=True,axis=1)

**Identify its category**

In [ ]:
data['category_info']=data.iloc[:,[1,2,3,5]].apply(lambda x:' '.join(x.astype(str)),axis=1)

In [ ]:
for i in range(len(data)):
    if bool(re.search(r'(pumps?|stilettos?|heels|boot(?:ie)?s?wedges?|sandals?|flats?|platform|mary\sjanes?|oxfords?|mules?|rainers?|loafers?|fantasys?|fantasies|shoes?|sneakers?)',
                    data.loc[i,'category_info'],flags=re.IGNORECASE))==True:
        data.loc[i,'category']='Shoe'
    elif bool(re.search(r'((?:sweat)pants?|jeans?|skirts?|shorts?|leggings?|joggers?|bottoms?)',
                    data.loc[i,'category_info'],flags=re.IGNORECASE))==True:
        data.loc[i,'category']='Bottom'  
    elif bool(re.search(r'(one(?:-|\s)?piece|overalls?|(?:jump|play|body|swim)suits?|rompers?|onesies?|unitards?|dungarees?|dress(?:es)?)',
                    data.loc[i,'category_info'],flags=re.IGNORECASE))==True:
        data.loc[i,'category']='One Piece'  
    elif bool(re.search(r'((?:hand)?bags?|satchels?|clutch(?:es)?|minaudieres?|wristlets?)',
                    data.loc[i,'category_info'],flags=re.IGNORECASE))==True:
        data.loc[i,'category']='Handbag'
    elif bool(re.search(r'(scarfs?|scarves)',
                    data.loc[i,'category_info'],flags=re.IGNORECASE))==True:
        data.loc[i,'category']='Scarf' 
    else:
        data.loc[i,'category']='null'

**Identify its color**

In [ ]:
data['color']=data['category_info'].str.findall(r'(beige|black|blue|brown|burgundy|gold|gray|green|navy|neutral|orange|pinks|purple|red|silver|teal|white|yellow)',flags=re.IGNORECASE)
data['colors']=data['color'].apply(lambda x:'Multi' if len(x)>1 else x)

In [ ]:
data.drop('color',axis=1, inplace=True)
data.drop('category_info',axis=1, inplace=True)

In [ ]:
data